In [ ]:
!pip install xgboost
!pip install s3fs

In [1]:
import matplotlib.pyplot as plt
import s3fs
import pandas as pd
import numpy as np
from itertools import product
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split,train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score,mean_squared_error, mean_absolute_error, r2_score,confusion_matrix
from sklearn.preprocessing import MinMaxScaler

In [2]:
def train_xgboost_model(X_train, X_test, y_train, y_test):
    # Split the data into training and testing sets

    # Define the XGBoost model
    model = xgb.XGBRegressor(objective="reg:squarederror")

    # Define the hyperparameter grid for grid search
    param_grid = {
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5],
        'n_estimators': [50, 100, 200],
    }

    # Use GridSearchCV to find the best hyperparameters
    grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=3)
    grid_search.fit(X_train, y_train)

    # Get the best model from the grid search
    best_model = grid_search.best_estimator_

    # Make predictions on the test set using the best model
    predictions = best_model.predict(X_test)

    # Calculate evaluation metrics
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    std_dev = np.std(y_test - predictions)  # Standard deviation of residuals

    return {
        "y_test": y_test,
        "predictions": predictions,
        "mse": mse,
        "mae": mae,
        "r2": r2,
        "std_dev": std_dev,
        "best_params": grid_search.best_params_
    }

In [20]:
bucket_name = 'sagemaker-studio-905418013525-nvxe84zgs6'
file_path = 'Labrado/rl8/rl8_Pot.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
rawdata1 = pd.read_csv(obj['Body'])

In [81]:
rawdata.shape

(2078644, 754)

In [35]:
pd.to_numeric(rawdata['50882'], errors='coerce').isna().sum()

7021

In [36]:
pd.to_numeric(rawdata1['Bic'], errors='coerce').isna().sum()

3

In [6]:
import boto3
s3 = boto3.client('s3')
itemid = 50971
bucket_name = 'sagemaker-studio-905418013525-nvxe84zgs6'
file_path = f"Labrado/alllabs1000adm/alllabs1000adm_{itemid}.csv"
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
rawdata2 = pd.read_csv(obj['Body'])

In [53]:
rawdata.head()

,hadm_id,chartyear,valuenum,charttime_diff_hours,50971,50971_dt,50983,50983_dt,50902,50902_dt,...,51088,51088_dt,51795,51795_dt,51068,51068_dt,51105,51105_dt,50881,50881_dt
0,20000019,2159,3.5,0,3.7,24.0,137.0,0.0,103.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20000019,2159,3.7,24,3.5,24.0,136.0,0.0,100.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20000019,2159,3.9,49,3.7,25.0,138.0,0.0,102.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20000024,2151,5.2,0,NaN,NaN,140.0,0.0,104.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20000034,2174,4.8,0,4.3,23.0,141.0,0.0,106.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
rawdata.columns[4::2]

Index(['50971', '50983', '50902', '51221', '50912', '50882', '50868', '51006',
       '50931', '51265',
       ...
       '51505', '51056', '51114', '51946', '51945', '51088', '51795', '51068',
       '51105', '50881'],
      dtype='object', length=375)

In [64]:
feature_cols = rawdata.columns[4::2]
target_col = rawdata.columns[2]
correlation = rawdata[feature_cols].corrwith(rawdata[target_col])
# Sort the absolute correlation values in descending order
correlation = correlation.abs().sort_values(ascending=False)
# Select the top features based on correlation coefficient
top_features = correlation.index.tolist()

/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


In [ ]:
condition = rawdata['chartyear'] <= 2178
TrainData = rawdata[condition].iloc[:99999,]
TestData = rawdata[~condition].iloc[:99999,]
feature_cols = rawdata.columns[4:10]
target_col = rawdata.columns[2]
X_train = TrainData[feature_cols]
y_train = TrainData[target_col]
X_test = TestData[feature_cols]
y_test = TestData[target_col]

In [85]:
print("Current time:", datetime.now())
model_results = train_xgboost_model(X_train, X_test, y_train, y_test)
print("Current time:", datetime.now())
print(model_results)

Current time: 2024-05-12 19:14:03.020244


ValueError: 
All the 81 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
81 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.10/site-packages/xgboost/core.py", line 620, in inner_f
    return func(**kwargs)
  File "/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py", line 988, in fit
    train_dmatrix, evals = _wrap_evaluation_matrices(
  File "/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py", line 448, in _wrap_evaluation_matrices
    train_dmatrix = create_dmatrix(
  File "/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py", line 908, in _

In [24]:
y_last = AllData[LabName].iloc[model_results['y_test'].index]
y_test = model_results['y_test']
y_pred = model_results['predictions']
valid_pos = y_last.notna()


mse_last = mean_squared_error(y_test[valid_pos], y_last[valid_pos])
r2_last = r2_score(y_test[valid_pos], y_last[valid_pos])
mse_pred = mean_squared_error(y_test[valid_pos], y_pred[valid_pos])
r2_pred = r2_score(y_test[valid_pos], y_pred[valid_pos])
print(LabName)
print("nearest neighbour mse ",round(mse_last,3),"and r2: ",round(r2_last,3))
print("xgboost imputation mse ",round(mse_pred,3),"and r2: ", round(r2_pred,3))



normalranges = {
  "Crt": (0.7, 1.3),
  "Plt": (150,450),
  "Hgb": (12, 18),
  "Wbc": (4, 11),
  "Ure": (8, 20),
  "Sod": (136, 145),
  "Bic": (23, 28),
  "Pot": (3.5, 5),
}

bins = [-1,normalranges[LabName][0],normalranges[LabName][1],10000]
# Define labels for the categories
labels = ['Low', 'Medium', 'High']
y_test_cat = pd.cut(y_test[valid_pos],bins=bins, labels=labels).astype(str)
y_last_cat = pd.cut(y_last[valid_pos],bins=bins, labels=labels).astype(str)
y_pred_cat = pd.cut(y_pred[valid_pos],bins=bins, labels=labels).astype(str)
print(LabName)
print(bins)
print("nearest neighbour mse ",round(mse_last,3),"and r2: ",round(r2_last,3))
print("xgboost imputation mse ",round(mse_pred,3),"and r2: ", round(r2_pred,3))
cm_last = confusion_matrix(y_test_cat,y_last_cat,labels=labels), 
print("confusion matrix for nearest neighbor")
print(cm_last)
cm_pred = confusion_matrix(y_test_cat,y_pred_cat,labels=labels)
print("confusion matrix for xgboost")
print(cm_pred)

Current time: 2024-05-12 15:35:55.321086
Current time: 2024-05-12 15:36:16.522628
{'y_test': 9        30.0
10       20.0
11       31.0
47       28.0
48       27.0
         ... 
53442    19.0
53474    30.0
53475    22.0
53476    30.0
53477    30.0
Name: valuenum, Length: 10000, dtype: float64, 'predictions': array([30.35885 , 29.828495, 29.6162  , ..., 24.077116, 27.607758,
       29.150644], dtype=float32), 'mse': 5.528349501477419, 'mae': 1.7958865118980407, 'r2': 0.7104673678252544, 'std_dev': 2.351202627127291, 'best_params': {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}}


NameError: name 'AllData' is not defined

In [6]:
print(LabName)
print(bins)
print("nearest neighbour mse ",round(mse_last,3),"and r2: ",round(r2_last,3))
print("xgboost imputation mse ",round(mse_pred,3),"and r2: ", round(r2_pred,3))
cm_last = confusion_matrix(y_test_cat,y_last_cat,labels=labels), 
print("confusion matrix for nearest neighbor")
print(cm_last)
cm_pred = confusion_matrix(y_test_cat,y_pred_cat,labels=labels)
print("confusion matrix for xgboost")
print(cm_pred)

Crt
[-1, 0.7, 1.3, 10000]
nearest neighbour mse  0.213 and r2:  0.918
xgboost imputation mse  0.151 and r2:  0.942
confusion matrix for nearest neighbor
(array([[ 87894,  13386,     75],
       [ 13187, 121787,   8296],
       [    66,   7402, 106249]]),)
confusion matrix for xgboost
[[ 73341  27944     70]
 [  5141 126140  11989]
 [    33   5214 108470]]
